In [1]:
import requests
import string
from datetime import datetime
from datetime import timedelta

from lxml import etree
from lxml import html

from pandas import DataFrame as df

In [2]:
currentDate = datetime.now()
yyyy = currentDate.year
mm = currentDate.month
dd = currentDate.day

dayback =timedelta(days=1, hours=22)
dateToExtact = currentDate - dayback

ext_yyyy = dateToExtact.year
ext_mm = dateToExtact.month
ext_dd = dateToExtact.day

queryParam = {'fromDate': ext_yyyy.__str__() + '-' + ext_mm.__str__() + '-' + ext_dd.__str__()}

queryParam

{'fromDate': '2016-1-4'}

In [4]:

proxies = {'http': 'proxy.wellsfargo.com:8080', 'https': 'proxy.wellsfargo.com:8080'}
parser = etree.HTMLParser()

# r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam, proxies=proxies)
r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam)

# r_dep = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_depinfo.do', params=queryParam, proxies=proxies)
r_dep = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_depinfo.do', params=queryParam)

r_dep.url

u'http://www.hongkongairport.com/flightinfo/chi/real_depinfo.do?fromDate=2016-1-4'

In [5]:
flightTable = []

tree = etree.fromstring(r.content, parser=parser)
trList =tree.xpath('//table[@id=\'list\']/tr[not(@mr)]')

for tr in trList:
    if 'date' in tr.attrib:
        dateStr = tr.attrib['date'].split(',')[1]
        yyyy=int(dateStr.split('-')[0])
        mm=int(dateStr.split('-')[1])
        dd=int(dateStr.split('-')[2])
    
    scheduledTimeStr = tr.getchildren()[0].text
    schHH = int(scheduledTimeStr.split(':')[0])
    schMM = int(scheduledTimeStr.split(':')[1])
    
    scheduledTime = datetime(yyyy, mm, dd, schHH, schMM)
    
    flightCode = tr.getchildren()[1].text
    
    city = tr.getchildren()[2].text
    
    airline = ''
    airlineRaw = tr.getchildren()[3].text
    for airlineStr in airlineRaw.split(','):
        if airlineStr.find('wm') == -1:
            airline = airline + ',' + airlineStr
    airline = airline[1:]
    
    gate = tr.getchildren()[4].text
    
    arriveTimeStr = tr.getchildren()[5].text
    if arriveTimeStr.find(':') == -1 :
        arriveTime = None
    elif arriveTimeStr.find('(') == -1 :
        timeStr = arriveTimeStr.split(' ')[1]
        arriveTime = datetime(yyyy, mm, dd, int(timeStr.split(':')[0]), int(timeStr.split(':')[1]))
    else :
        dateStr = arriveTimeStr.split('(')[1].split(')')[0]
        timeStr = arriveTimeStr.split(' ')[1]
        
        yyyyActual = int(dateStr.split('/')[2])
        mmActual = int(dateStr.split('/')[1])
        ddActual = int(dateStr.split('/')[0])
        
        hourActual = int(timeStr.split(':')[0])
        minActual = int(timeStr.split(':')[1])
        
        arriveTime = datetime(yyyyActual, mmActual, ddActual, hourActual, minActual)
        
    flightEntry = {'scheduledTime': scheduledTime, 
                  'flightCode': flightCode,
                  'city': city,
                  'airline': airline,
                  'gate': gate,
                  'arriveTime': arriveTime}
    
    flightTable.append(flightEntry)
    
flightData = df(flightTable)

flightData.head()


,airline,arriveTime,city,flightCode,gate,scheduledTime
0,香港快運航空,2016-01-04 01:04:00,台中/清泉崗,UO 165,B,2016-01-03 21:55:00
1,"大韓航空,美國達美航空公司",2016-01-04 02:11:00,首爾/仁川,"KE 607,DL 7897",B,2016-01-03 22:55:00
2,"港龍航空公司,國泰航空公司",2016-01-04 01:48:00,上海/浦東,"KA 809,CX 5809",B,2016-01-03 23:25:00
3,中華航空,2016-01-04 00:27:00,台北,CI 677,B,2016-01-03 23:50:00
4,香港快運航空,2016-01-04 00:01:00,清邁,UO 754,B,2016-01-03 23:50:00


In [6]:
flightData[flightData.arriveTime.isnull()]

,airline,arriveTime,city,flightCode,gate,scheduledTime
30,香港快運航空,NaT,大阪/關西,UO 1671,,2016-01-04 05:55:00
66,美佳航空,NaT,馬累,LV 136,,2016-01-04 08:20:00


In [7]:
r_dep.text

u'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n<script type="text/javascript">\r\n//Get cache time in real_depinfo \r\nvar noRec=\'\u6c92\u6709\u8cc7\u6599\';\r\nvar widths=["36", "59", "95", "28", "28", "26", "131", "82"];\r\nvar types=[null, \'FN\', \'AP\', null, null, null, \'AL\', null];\r\nvar aligns=[\'center\', null, null, \'center\', \'center\', \'center\', null, null];\r\ntdClasses=["flightContent", "flightNum", "flightContent", "flightContentCenter", "flightContentCenter", "flightContentCenter", "flightLogoAndContent", "flightContentEnd"];\r\n</script>\r\n<script type="text/javascript" src="../include/list.js"></script>\r\n<script language="javascript" type="text/javascript">\r\n\tvar section = 0;\r\n\tvar level1 = "flight";\r\n\tvar level2 = "real";\r\n\tvar level3 = "